In [1]:
import warnings
warnings.filterwarnings('ignore')
import re
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_20newsgroups, load_files
import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from tqdm.auto import tqdm
import json
from fractions import Fraction

In [2]:
# df_orig = pd.read_csv('./data/data.csv')

In [3]:
def contains_digits(s: str):
    """Функция для проверки наличия цифр

    Args:
        s (str): строка

    Returns:
        bool: наличие или отсутствие цифр в строке
    """
    return any(char.isdigit() for char in s)

def code_rare_categories(df, column: str, threshold: float, new_val):
    """кодирование редких категорий как other, используя threshold
    
    Keyword arguments: \n
    df -- dataframe\n
    column -- имя колонки\n
    threshold -- процентный допуск, ниже которого ставится значение other\n
    Return: обновленный df
    """
    
    category_counts = df[column].value_counts()
    total_count = len(df)
    category_percentages = (category_counts / total_count) * 100
    rare_categories = category_percentages[category_percentages < threshold].index
    new_column = column + '_cleaned'
    df[new_column] = df[column].apply(lambda x: new_val if x in rare_categories else x)
    return df

# end def

def replace_based_on_pattern(value, pattern, new_value):
    if re.search(pattern, value):
        return new_value
    return value


def convert_target(cell_val):
    """
    Purpose: 
    """
    pattern = r"\d+(,\d+)?(\.\d+)?"
    
    num_str = re.sub(r'[^0-9.]', '', cell_val)

    # num_str = re.search(pattern, cell_val)

    if (num_str):
        # comment: 
        try:
            # comment: 
            return float(num_str)
        except Exception as e:
            return 0
        # end try
        # return float(num_str)
    else:
        return None
        # comment: 
    # end if
# end def

def addr_to_coords(addr, geolocator):
    """
    Purpose: 
    """
    geolocator = Nominatim(user_agent="my_geocoder", scheme='http', domain='localhost:8080', timeout=10)

    # Геокодирование адреса
    try:
        # comment: 
        location = geolocator.geocode(addr)
        if (location):
        # comment:
            return (location.latitude, location.longitude)
        else:
            # comment: 
            return None
    # end if
    except (GeocoderTimedOut, GeocoderUnavailable) as e:
        print(f"Error: {e}. Retrying...")
        time.sleep(2)  # Добавляем задержку перед повторной попыткой
        return addr_to_coords(addr, geolocator)
    # end try

def convert_bath(cell_val):
    """
    Purpose: 
    """
    pattern = r"\d+[,.]?\d*"
    num_str = re.search(pattern, cell_val)

    if (num_str):
        # comment: 
        return float(num_str.group().replace(',' , ''))
    else:
        return None
        # comment: 
    # end if
# end def

def value_percentage(df, column):
    """
    Purpose: процент значений колонки от общего числа строк
    """
    name_series = df[column].value_counts(dropna=False)
    name_df = name_series.to_frame()
    name_df.reset_index(inplace=True)
    name_df['percent'] = name_df['count'] / df.shape[0] * 100
    return name_df

# end def

# Пользовательская функция для объединения столбцов по условию
def combine_columns(row):
    if ((row['private_pool'] + row['privatepool']) < 1):
        return 0
    else:
        return 1
    
def missing_values_per_column(df):
    """
    Purpose: вывод количества отсутствующих значений в датасете
    """
    missing_values = df.isna().sum()
    print("Количество значений NaN по каждой колонке:")
    print(missing_values)

# end def

# Функция для безопасной десериализации JSON-строк
def safe_json_loads(json_str):
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return None
    

# Функция для извлечения данных из 'atAGlanceFacts'
def extract_home_facts(home_facts, label):
    if not home_facts or 'atAGlanceFacts' not in home_facts:
        return None
    for fact in home_facts['atAGlanceFacts']:
        if fact.get('factLabel') == label:
            return fact.get('factValue')
    return None



def quote_replace(cell_val):
    
    def replace(match):
        return f'{match.group(1)}\'{match.group(3)}'

    """
    Purpose: 
    """
    pattern = r'([a-zA-Z])(\")([a-zA-Z])'
    new_text = re.sub(pattern, replace, cell_val)

    return new_text
# end def

def word_in_quote_replace(cell_val):
    
    def replace(match):
        return f'\'{match.group(2)}\''

    """
    Purpose: 
    """
    pattern = r'(\")(\w+)(\")'
    new_text = re.sub(pattern, replace, cell_val)

    return new_text
# end def

def get_school_rating(cell_val):
    """
    Purpose: 
    """
    if (cell_val):
        # comment: 
        rating = cell_val.get('rating')

        if (rating):
            # comment:
            av_rate = []
            for item in rating:
                # comment: 
                try:
                    # comment: 
                    item = float(Fraction(item))
                    av_rate.append(item)
                except Exception as e:
                    item = 0
                    av_rate.append(item)
                # end try
            # end for
            return round(np.mean(av_rate), 2)
        else:
            # comment: 
            return None
        # end if
    else:
        return None
        # comment: 
    # end if    
# end def

def get_schools_qty(cell_val):
    """
    Purpose: 
    """
    if (cell_val):
        # comment: 
        rating = cell_val.get('rating')

        if (rating):
            return len(rating)
        else:
            # comment: 
            return None
        # end if
    else:
        return None
        # comment: 
    # end if    
# end def

def get_avg_school_dist(cell_val):
    """
    Purpose: 
    """
    if (cell_val):
        # comment: 
        rating = cell_val.get('data')

        if (rating):
            # comment:
            dist = rating.get('Distance')
            dist_list = []
            for item in dist:
                # comment: 
                pattern = r"[-+]?\d*[.,]\d+"
                if (re.search(pattern, item)):
                    # comment: 
                    number = re.search(pattern, item).group()
                else:
                    # comment: 
                    number = 0
                # end if
                dist_list.append(float(number))
            # end for
            return round(np.mean(dist_list), 2)
        else:
            # comment: 
            return None
        # end if
    else:
        return None
        # comment: 
    # end if    
# end def



df = df_orig.copy()

df.drop(['zipcode', 'mls-id', 'MlsId'], axis=1, inplace=True)

df['street'] = df['street'].astype(str).str.strip()
df['state'] = df['state'].astype(str).str.strip()
df['full_addr'] = df['street'] +', '+ df['state']

tqdm.pandas(desc="Processing rows")

df100 = df.copy()

df100['coords'] = df100['full_addr'].progress_apply(lambda x: addr_to_coords(x, geolocator))
df100[['latitude', 'longitude']] = df100['coords'].apply(lambda x: pd.Series(x) if x is not None else pd.Series([None, None]))

df100.to_csv('./data/data_with_coords50000.csv', mode='a+', header=False)

In [4]:
df = pd.read_csv('data/data_with_coords50000.csv', index_col=0)
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(' ', '_')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 377185 entries, 0 to 377184
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        337267 non-null  object 
 1   private_pool  4181 non-null    object 
 2   propertytype  342452 non-null  object 
 3   street        377183 non-null  object 
 4   baths         270847 non-null  object 
 5   homefacts     377185 non-null  object 
 6   fireplace     103114 non-null  object 
 7   city          377151 non-null  object 
 8   schools       377185 non-null  object 
 9   sqft          336608 non-null  object 
 10  beds          285903 non-null  object 
 11  state         377185 non-null  object 
 12  stories       226469 non-null  object 
 13  privatepool   40311 non-null   object 
 14  target        374704 non-null  object 
 15  full_addr     377185 non-null  object 
 16  coords        264625 non-null  object 
 17  latitude      264625 non-null  float64
 18  longitude

Изменим имена колонок

In [6]:
df = df.rename(columns={
    'propertytype': 'property_type',
    'latitude': 'lat',
    'longitude': 'lon'
})

Удалим колонки, участвовавшие в геокодировании и строки с отсутствующими координатами

In [7]:
df = df.dropna(subset=['coords'])

labels = [
    'street',
    'city',
    'full_addr',
    'coords'
]

df.drop(labels=labels, axis=1, inplace=True)


In [8]:
missing_values_per_column(df)

Количество значений NaN по каждой колонке:
status            25067
private_pool     261609
property_type     20641
baths             71332
homefacts             0
fireplace        177950
schools               0
sqft              27122
beds              58205
state                 0
stories           97523
privatepool      239905
target             2103
lat                   0
lon                   0
dtype: int64


точно можно удалить строки с нулевыми значениями в target, толку от неизвестной цены не будет

In [9]:
df = df.dropna(subset=['target'])

df['target'] = df['target'].astype(str)
df['target'] = df['target'].map(convert_target)

Обработаем столбец 'private_pool'

In [10]:
df.loc[df['private_pool'] == 'Yes', 'private_pool'] = 1
df['private_pool'].fillna(0, inplace=True)
df['private_pool'] = df['private_pool'].astype(int)

Обработаем столбец 'privatepool'

In [11]:
df.loc[df['privatepool'] == 'Yes', 'privatepool'] = 1
df.loc[df['privatepool'] == 'yes', 'privatepool'] = 1
df['privatepool'].fillna(0, inplace=True)
df['privatepool'] = df['privatepool'].astype(int)

Объединим столбцы 'private_pool' и 'privatepool'

In [12]:
df['pool'] = df.apply(combine_columns, axis=1)
df.drop(['private_pool', 'privatepool'], axis=1, inplace=True)

Приведем статус for sale к единому виду

In [13]:
df.loc[df['status'] == 'For sale', 'status'] = 'for_sale'
df.loc[df['status'] == 'for sale', 'status'] = 'for_sale'

Посмотрим, какие значения есть в колонке status и как они распределяются

In [14]:
status_df = value_percentage(df, 'status')
status_df

,status,count,percent
0,for_sale,139938,53.305247
1,Active,74157,28.247918
2,NaN,24558,9.354645
3,foreclosure,5059,1.927077
4,Pending,3974,1.513778
...,...,...,...
144,Coming soon: Oct 29.,1,0.000381
145,Pending W/Backup Wanted,1,0.000381
146,Pending Backups Wanted,1,0.000381
147,Pending Bring Backup,1,0.000381


Оставим объекты, которые в данный момент активны и продаются

In [15]:
filtered_status_df = status_df.loc[status_df['percent'] >= 10]
filtered_status_df

,status,count,percent
0,for_sale,139938,53.305247
1,Active,74157,28.247918


In [16]:
status_list = filtered_status_df['status'].to_list()
status_list

['for_sale', 'Active']

In [17]:
df = df.loc[df['status'].isin(status_list)]

Удалим колонки for_sale и Active, они не нужный, в таблице только активные объекты

In [18]:
df.drop(['status'], axis=1, inplace=True)

Закодируем статус объекта недвижимости

df = pd.get_dummies(df, columns=['status'], dtype=int)

In [19]:
missing_values_per_column(df)

Количество значений NaN по каждой колонке:
property_type     17763
baths             63410
homefacts             0
fireplace        142298
schools               0
sqft              23098
beds              55929
state                 0
stories           86262
target                0
lat                   0
lon                   0
pool                  0
dtype: int64


In [20]:
code_rare_categories(df=df, column='baths', threshold=0.02, new_val=0)
df['baths_cleaned'] = df['baths_cleaned'].fillna(0)

In [21]:
_df = value_percentage(df=df, column='baths_cleaned')
_df.to_csv('./data/baths.csv')

In [22]:
df['baths_cleaned'] = df['baths_cleaned'].astype(str)
df['baths_cleaned'] = df['baths_cleaned'].apply(convert_target)
df['baths_cleaned'] = df['baths_cleaned'].fillna(0)

df.drop(labels=['baths'], axis=1, inplace=True)

df = df.rename(columns={
    'baths_cleaned': 'baths',
})

In [23]:
df['property_type'] = df['property_type'].str.lower()
df['property_type'] = df['property_type'].astype(str)

In [24]:
pattern = r'single.*family'  # Шаблон для поиска
new_value = 'single_family'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))


In [25]:
pattern = r'mult.*family'  # Шаблон для поиска
new_value = 'multi_family'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))


In [26]:
pattern = r'land'  # Шаблон для поиска
new_value = 'land'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [27]:
pattern = r'condo'  # Шаблон для поиска
new_value = 'condo'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [28]:
pattern = r'ranch'  # Шаблон для поиска
new_value = 'ranch'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [29]:
pattern = r'nan'  # Шаблон для поиска
new_value = 'other'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [30]:
pattern = r'detached'  # Шаблон для поиска
new_value = 'single_family'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

Обработаем категории с целью уменьшения списка

Найдем редкие категории и назовем их other

In [31]:
df = code_rare_categories(df=df, column='property_type', threshold=1, new_val='other')
df.drop(labels=['property_type'], axis=1, inplace=True)
df = df.rename(columns={
    'property_type_cleaned': 'property_type'
})

In [32]:
_df = value_percentage(df, 'property_type')
_df.head(50)

,property_type,count,percent
0,single_family,127949,59.762722
1,other,31075,14.514585
2,land,24249,11.326280
3,multi_family,9390,4.385903
4,townhouse,8853,4.135080
5,condo,5028,2.348490
6,traditional,4817,2.249936
7,ranch,2734,1.277003


In [33]:
df = pd.get_dummies(df, columns=['property_type'], dtype=int)

In [34]:
df['fireplace'] = df['fireplace'].fillna(0)
df['fireplace'] = df['fireplace'].astype(str)
code_rare_categories(df=df, column='fireplace', threshold=0.1, new_val='other')

,homefacts,fireplace,schools,sqft,beds,state,stories,target,lat,lon,...,baths,property_type_condo,property_type_land,property_type_multi_family,property_type_other,property_type_ranch,property_type_single_family,property_type_townhouse,property_type_traditional,fireplace_cleaned
0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,4,NC,NaN,418000.0,35.627887,-77.335173,...,3.5,0,0,0,0,0,1,0,0,other
1,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",0,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",3 Beds,WA,2.0,310000.0,47.699482,-117.190979,...,3.0,0,0,0,0,0,1,0,0,0
2,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",3 Beds,CA,1.0,2895000.0,33.943243,-117.376310,...,2.0,0,0,0,0,0,1,0,0,yes
3,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",5 Beds,TX,3.0,2395000.0,32.702323,-97.347071,...,8.0,0,0,0,0,0,1,0,0,yes
4,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",0,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,NaN,FL,NaN,5000.0,27.041567,-82.136146,...,0.0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377173,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",yes,"[{'rating': ['6/10', '5/10', '6/10'], 'data': ...","2,505 sqft",5 Beds,OH,2.0,384900.0,41.456416,-81.836809,...,3.0,0,0,0,0,0,1,0,0,yes
377174,"{'atAGlanceFacts': [{'factValue': '1938', 'fac...",0,"[{'rating': ['4/10', '5/10', '3/10'], 'data': ...",950 sqft,2 Beds,TX,1.0,799500.0,29.441120,-98.487583,...,0.0,0,0,0,0,0,1,0,0,0
377175,"{'atAGlanceFacts': [{'factValue': '1970', 'fac...",yes,"[{'rating': ['2/10', '3/10', '3/10'], 'data': ...","1,792 sqft",4 Beds,TX,2.0,280000.0,29.826059,-95.531518,...,3.0,0,0,0,0,0,1,0,0,yes
377177,"{'atAGlanceFacts': [{'factValue': '1921', 'fac...",0,"[{'rating': ['4', '6', '5'], 'data': {'Distanc...","1,895",NaN,TX,NaN,199900.0,32.711046,-97.330033,...,0.0,0,0,0,0,0,1,0,0,0


In [35]:
pattern = r'Not Applicable'  # Шаблон для поиска
new_value = '0'  # Новое значение для замены

df['fireplace'] = df['fireplace'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [36]:
df.loc[(df['fireplace_cleaned'] != '0') & (df['fireplace_cleaned'] != 'other'), 'fireplace_cleaned'] = '1'

In [37]:
df.drop(labels=['fireplace'], axis=1, inplace=True)
df = df.rename(columns={
    'fireplace_cleaned': 'fireplace'
})

In [38]:
df = pd.get_dummies(df, columns=['fireplace'], dtype=int)

In [39]:
df.sample(5, random_state=48)

,homefacts,schools,sqft,beds,state,stories,target,lat,lon,pool,...,property_type_land,property_type_multi_family,property_type_other,property_type_ranch,property_type_single_family,property_type_townhouse,property_type_traditional,fireplace_0,fireplace_1,fireplace_other
347530,"{'atAGlanceFacts': [{'factValue': '1973', 'fac...","[{'rating': ['3', '3', '1', '2', '2', 'NR', 'N...",1591,3.0,TN,1.00,225000.0,36.053767,-86.674700,0,...,0,0,0,0,1,0,0,0,0,1
281121,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...","[{'rating': ['4/10', 'None/10', '9/10', '6/10'...","1,740 sqft",3 Beds,OR,2.0,333900.0,45.391952,-122.609684,0,...,0,0,0,0,0,0,0,1,0,0
130895,"{'atAGlanceFacts': [{'factValue': '1988', 'fac...","[{'rating': ['9/10', '6/10', '8/10', '9/10'], ...","1,803 sqft",3 Beds,TX,1.0,205000.0,29.578067,-95.378902,0,...,0,0,0,0,1,0,0,0,1,0
106801,"{'atAGlanceFacts': [{'factValue': '1972', 'fac...","[{'rating': ['1', 'NR', '3', '4', '4'], 'data'...","1,531",Baths,AZ,NaN,219500.0,33.491844,-112.244938,0,...,0,0,0,1,0,0,0,1,0,0
166646,"{'atAGlanceFacts': [{'factValue': '', 'factLab...","[{'rating': ['5/10', '4/10'], 'data': {'Distan...","2,566 sqft",3 Beds,TX,NaN,224855.0,31.680224,-106.254632,0,...,0,0,0,0,1,0,0,1,0,0


Обработаем колонку sqft

In [40]:
df['sqft'] = df['sqft'].fillna(0)
df['sqft'] = df['sqft'].astype(str)
df['sqft'] = df['sqft'].map(convert_target)
df['sqft'] = df['sqft'].fillna(0)
df['sqft'] = df['sqft'].astype(int)

Обработаем колонку states

In [41]:
df_states = pd.read_csv('./data/states.csv', header=None, names=['State', 'Code'])
state_to_code = df_states.set_index('State')['Code'].to_dict()

df['state_code'] = df['state'].map(state_to_code)
df.drop(labels=['state'], axis=1, inplace=True)
df = df.rename(columns={
    'state_code': 'state'
})

Обработаем колонку beds

In [42]:
df['beds'] = df['beds'].fillna(0)
df['beds'] = df['beds'].astype(str)
df['beds'] = df['beds'].apply(lambda x: x if contains_digits(x) else '0')
df['beds'] = df['beds'].str.replace(r'[^0-9.]+', '', regex=True)
df['beds'] = pd.to_numeric(df['beds'], errors='coerce')
df['beds'] = df['beds'].fillna(0)
df['beds'] = df['beds'].astype(int)
df['beds'] = np.where(df['beds'] > 5, 0, df['beds'])

Обработаем колонку homefacts

In [43]:
df['homefacts'] = df['homefacts'].str.replace('\'', '\"')
df['homefacts'] = df['homefacts'].map(quote_replace)
df['homefacts'] = df['homefacts'].str.replace('\"factValue": None', '"factValue": "None"')

# Десериализация JSON-полей
df['homefacts_deserialized'] = df['homefacts'].apply(safe_json_loads)
df['glance'] = df['homefacts_deserialized'].apply(lambda x: x['atAGlanceFacts'] if x is not None else None)

df['year_built'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'Year built'))
df['remodeled_year'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'Remodeled year'))
df['heating'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'Heating'))
df['cooling'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'Cooling'))
df['parking'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'Parking'))
df['lot_size'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'lotsize'))
df['price_per_sqft'] = df['homefacts_deserialized'].apply(lambda x: extract_home_facts(x, 'Price/sqft'))

df.drop(['homefacts', 'homefacts_deserialized', 'glance'], axis=1, inplace=True)


Обработаем колонку schools

In [44]:
df['schools'] = df['schools'].str.replace('\'', '\"')
df['schools'] = df['schools'].map(quote_replace)
df['schools_deserialized'] = df['schools'].apply(safe_json_loads)
df['schools_deserialized'] = df['schools_deserialized'].map(lambda x: x[0] if x else None)

df['avg_school_rating'] = df['schools_deserialized'].map(get_school_rating)
df['avg_school_rating'] = df['avg_school_rating'].fillna(0)

df['schools_qty'] = df['schools_deserialized'].map(get_schools_qty)
df['schools_qty'] = df['schools_qty'].fillna(0)
df['schools_qty'] = df['schools_qty'].astype(int)

df['avg_school_dist'] = df['schools_deserialized'].map(get_avg_school_dist)
df['avg_school_dist'] = df['avg_school_dist'].fillna(0)

df.drop(['schools', 'schools_deserialized'], axis=1, inplace=True)

In [45]:
df.to_csv('data/df_cleaned.csv')

In [46]:
_df = value_percentage(df=df, column='stories')
_df.head(15)


,stories,count,percent
0,NaN,86262,40.291459
1,1.0,40989,19.145239
2,2.0,34304,16.022794
3,1,11874,5.546136
4,2,10723,5.008524
5,3.0,5982,2.794087
6,0.0,3928,1.834700
7,One,3778,1.764637
8,3,2207,1.030851
9,Two,1931,0.901936


Почти 40% записей не имеет данных об этажности объекта, колонка stories полезной не будет

In [47]:
df.drop('stories', axis=1, inplace=True)

In [48]:
df.head()

,sqft,beds,target,lat,lon,pool,baths,property_type_condo,property_type_land,property_type_multi_family,...,year_built,remodeled_year,heating,cooling,parking,lot_size,price_per_sqft,avg_school_rating,schools_qty,avg_school_dist
0,2900,4,418000.0,35.627887,-77.335173,0,3.5,0,0,0,...,2019,,"Central A/C, Heat Pump",,,None,$144,3.25,8,5.54
1,1947,3,310000.0,47.699482,-117.190979,0,3.0,0,0,0,...,2019,,,,,5828 sqft,$159/sqft,0.27,3,1.33
2,3000,3,2895000.0,33.943243,-117.376310,1,2.0,0,0,0,...,1961,1967,Forced Air,Central,Attached Garage,"8,626 sqft",$965/sqft,0.67,3,1.96
3,6457,5,2395000.0,32.702323,-97.347071,0,8.0,0,0,0,...,2006,2006,Forced Air,Central,Detached Garage,"8,220 sqft",$371/sqft,0.92,4,0.75
4,0,0,5000.0,27.041567,-82.136146,0,0.0,0,1,0,...,,,,,,"10,019 sqft",None,0.47,3,4.08


In [49]:
value_percentage(df, 'remodeled_year')

,remodeled_year,count,percent
0,,115954,54.160069
1,None,5599,2.615194
2,2006,2753,1.285878
3,2005,2746,1.282608
4,2007,2095,0.978538
...,...,...,...
148,1800,1,0.000467
149,1869,1,0.000467
150,1897,1,0.000467
151,1874,1,0.000467


In [50]:
df['remodeled_year'] = df['remodeled_year'].apply(lambda x: x if x != '' else 0)
df['year_built'] = df['year_built'].apply(lambda x: x if x != '' else 0)

df['remodeled_year'] = df['remodeled_year'].apply(lambda x: 0 if ((x == 'None') or (x == 'No Data')) else x)
df['year_built'] = df['year_built'].apply(lambda x: 0 if ((x == 'None') or (x == 'No Data')) else x)

df['remodeled_year'] = df['remodeled_year'].astype(int)
df['year_built'] = df['year_built'].astype(int)

In [51]:
_df = value_percentage(df=df, column='parking')
_df.head(20)

,parking,count,percent
0,,102546,47.897429
1,Attached Garage,48451,22.630608
2,2 spaces,11107,5.187884
3,Detached Garage,9738,4.548448
4,1 space,4735,2.211635
5,Carport,4478,2.091595
6,No Data,2314,1.080829
7,3 spaces,2286,1.067750
8,Off Street,2234,1.043462
9,"Carport, Attached Garage",2160,1.008898


In [52]:
_df = value_percentage(df=df, column='heating')
_df.head(20)

,heating,count,percent
0,,60335,28.181415
1,Forced Air,56511,26.395292
2,Forced air,22401,10.463112
3,Other,17422,8.137509
4,Gas,5612,2.621266
5,Electric,5211,2.433966
6,Central Electric,4918,2.297111
7,Heat Pump,4332,2.023401
8,Central,4055,1.894019
9,Central Air,3389,1.582942


In [53]:
_df = value_percentage(df=df, column='cooling')
_df.head(20)

,cooling,count,percent
0,Central,89076,41.605829
1,,70896,33.114272
2,Central Air,8896,4.155165
3,None,5639,2.633877
4,Has Cooling,4668,2.180341
5,Central Electric,3543,1.654873
6,Central Gas,2955,1.380228
7,Wall,2167,1.012167
8,No Data,1898,0.886522
9,Central Heating,1677,0.783297


In [54]:
df.drop(['parking', 'heating', 'cooling', 'lot_size'], axis=1, inplace=True)

In [55]:
df['price_per_sqft'] = df['price_per_sqft'].map(convert_target)
df['price_per_sqft'] = df['price_per_sqft'].fillna(0)
df['price_per_sqft'] = df['price_per_sqft'].astype(float)

In [56]:
missing_values_per_column(df)

Количество значений NaN по каждой колонке:
sqft                           0
beds                           0
target                         0
lat                            0
lon                            0
pool                           0
baths                          0
property_type_condo            0
property_type_land             0
property_type_multi_family     0
property_type_other            0
property_type_ranch            0
property_type_single_family    0
property_type_townhouse        0
property_type_traditional      0
fireplace_0                    0
fireplace_1                    0
fireplace_other                0
state                          0
year_built                     0
remodeled_year                 0
price_per_sqft                 0
avg_school_rating              0
schools_qty                    0
avg_school_dist                0
dtype: int64


In [57]:
col = df.pop('target')

# Вставляем его в конец
df.insert(len(df.columns), 'target', col)

In [58]:
df.to_csv('./data/df_for_ML.csv')

In [59]:
df.head()

,sqft,beds,lat,lon,pool,baths,property_type_condo,property_type_land,property_type_multi_family,property_type_other,...,fireplace_1,fireplace_other,state,year_built,remodeled_year,price_per_sqft,avg_school_rating,schools_qty,avg_school_dist,target
0,2900,4,35.627887,-77.335173,0,3.5,0,0,0,0,...,0,1,37,2019,0,144.0,3.25,8,5.54,418000.0
1,1947,3,47.699482,-117.190979,0,3.0,0,0,0,0,...,0,0,53,2019,0,159.0,0.27,3,1.33,310000.0
2,3000,3,33.943243,-117.376310,1,2.0,0,0,0,0,...,1,0,6,1961,1967,965.0,0.67,3,1.96,2895000.0
3,6457,5,32.702323,-97.347071,0,8.0,0,0,0,0,...,1,0,48,2006,2006,371.0,0.92,4,0.75,2395000.0
4,0,0,27.041567,-82.136146,0,0.0,0,1,0,0,...,0,0,12,0,0,0.0,0.47,3,4.08,5000.0
